# **Prompt Engineering Guide**

This notebook demonstrates prompt engineering principles. It contains explanations, code examples (zero-shot and few-shot), tips on *task, context, examples, role, format, tone*, and guidance for using chain-of-thought (CoT) techniques.

# Setup

Make sure you set your `GROQ_API_KEY` in the environment before running any cells that call the API.

In [1]:
# Import packages
import os
from groq import Groq
from dotenv import load_dotenv
from IPython.display import Markdown, HTML

In [2]:
# Load GROQ API key from .env file
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

In [3]:
# Define a helper function around Groq's Chat Completion
client = Groq(api_key=api_key)

def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        messages=messages,
        model="openai/gpt-oss-120b",
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message.content

# Components of a Prompt 

## 1. Task

The explicit instruction stating what you want the model to do.

Keep it clear, specific, and focused on a single task at a time.

In [4]:
# Task-only (Zero-shot)
input_text = "Hi, we are learning Prompt Engineering."

prompt = f"""Translate the following input text into German.
Input text: {input_text}
"""

response = get_completion(prompt)

print(response)

Hallo, wir lernen Prompt Engineering.


## 2. Context

Any background information or constraints that help the model produce a more accurate result.

In [5]:
# Task + Context
prompt = """
Context: Pizza Planet is a small family-owned pizza shop in New York City. 
They are famous for their wood-fired pizzas and friendly service.

Task: Write a short and catchy Instagram post to attract customers.
"""

result = get_completion(prompt)
Markdown(result)

🍕✨ **Pizza Planet** – where NYC’s streets meet the perfect wood‑fire blaze! 🌟  
Come for the smoky, melt‑in‑your‑mouth slices, stay for the warm, family‑style vibes.  

🕒 Open daily 11 am‑10 pm  
📍 123 Main St, New York, NY  

📸 Snap your slice, tag us #PizzaPlanetNYC and get a **FREE garlic knot** on your next visit!  

🔥 **Wood‑fired. Family‑owned. Unforgettable.**  

#NYCPizza #WoodFirePizza #FamilyOwned #PizzaLovers #NYCEats 🍕💙

## 3. Examples (Zero-shot / Few-shot / Chain-of-Thought (CoT))

#### **Zero-shot:**
No examples. Rely on the task and context.

In [6]:
# Zero-shot example
prompt = "Explain how Wi-Fi works in simple words, as if you are talking to a 10-year-old."

Markdown(get_completion(prompt))

**Imagine a magical invisible road that lets your gadgets talk to each other without any wires. That’s what Wi‑Fi is!**  

---

### 1. The “Post Office” in Your Home  
- **Router** – Think of this as the post office in your house. It’s a little box that sits on a shelf or a table.  
- **What it does:** It receives messages (like emails, videos, game data) from the big internet “city” and then sends those messages to the devices in your house (your tablet, laptop, game console, etc.).  

### 2. The “Invisible Highway”  
- **Radio waves** – Just like the radio in the car that plays music, there are invisible waves that travel through the air. Wi‑Fi uses special radio waves that are too fast for us to see.  
- **How they work:** The router sends out these waves, and your devices catch them, kind of like catching a ball that’s thrown through the air.

### 3. How a Device Gets the Message  
1. **You ask for something** – You tap a picture on your tablet or click a video on YouTube.  
2. **Your device shouts, “Hey, I want that!”** – It sends a tiny request on the invisible highway to the router.  
3. **The router talks to the internet** – The router talks to the big internet “city” (through a cable or a big antenna) and asks for the picture or video.  
4. **The internet sends it back** – The internet sends the picture/video back to the router.  
5. **Router sends it to you** – The router turns the data into radio waves, and your device catches them and shows you the picture or video on the screen.

### 4. Why It’s “Wi‑Fi”  
- **Wi‑** comes from “wireless” (no wires).  
- **-Fi** comes from “Fidelity,” which is a fancy word for “good quality.” So “Wi‑Fi” means “wireless with good quality.”

### 5. A Quick Everyday Example  

| **Real‑world thing** | **Wi‑Fi version** |
|---------------------|-----------------|
| **Post office**   | Router (the house’s post office) |
| **Mail**        | Data (pictures, videos, game info) |
| **Roads**      | Invisible radio waves |
| **Delivery truck** | The signal that travels back and forth |
| **Your house** | Your home where all the devices live |

### 6. Why It’s Cool  
- **No wires needed** – You can move around with your tablet or phone and still stay connected.  
- **Many devices at once** – The router can talk to many gadgets at the same time, like a busy post office handling many letters.  
- **Fast** – The radio waves travel super‑fast, so you can watch videos or play games without waiting long.

### 7. A Little Safety Tip  
Just like you lock your front door so strangers can’t come in, you set a **password** on your Wi‑Fi. That way only the people you want can use your invisible road.

---

**In a nutshell:**  
Your router is a tiny, invisible post office that uses invisible radio waves (the invisible highway) to send and receive messages from the internet. Your devices catch those waves, and that’s how you can watch videos, play games, and chat with friends— all without any wires! 🎉

#### **Few-shot:**
Provide 1–3 short `input → output` examples so the model can infer the structure and style.

In [7]:
# Few-shot example
message = "I received the wrong size shirt. Can I exchange it?"

prompt = f"""You are a customer service assistant. 
Read the customer's email and reply politely, keeping your answer short and clear.

Examples:

Customer: "My package arrived late and the box was damaged."
Reply: "We’re sorry to hear that! Please send us a photo of the damage, and we’ll arrange a replacement right away."

Customer: "Can I change my delivery address?"
Reply: "Of course! Please reply with your new address, and we’ll update it before shipping."

Now complete this one:

Customer: {message}
Reply:
"""

print(get_completion(prompt))

We’re sorry about that! Please let us know your order number and the size you need, and we’ll arrange an exchange right away.


#### **Chain-of-Thought (CoT) Techniques**

Chain-of-thought prompts instruct the model to reveal intermediate reasoning steps. Use them carefully:

- **When to use:** debugging, teaching, transparent reasoning.
- **When to avoid:** tasks requiring concise or private outputs, or when you don't want the model to hallucinate extra steps.

**Controlled-CoT pattern (preferred):** Ask for a short step-by-step reasoning section followed by a final concise answer.

**Note:** Explicit CoT can sometimes increase risk of hallucination and token use. If the model is unreliable with CoT, use few-shot examples of the expected reasoning format instead.

In [8]:
# Cell: Controlled Chain-of-Thought example
prompt = """You are a financial advisor helping a small bakery decide if they can afford a new oven.

Step-by-step reasoning instructions:
1) Calculate current monthly profit:
   - Average monthly revenue = $12,500
   - Average monthly expenses (ingredients, wages, utilities) = $9,200
2) The new oven costs $4,800 upfront and will reduce monthly ingredient waste by $250.
3) The bakery wants to pay off the oven within 12 months using only the extra savings from reduced waste.
4) Determine:
   a) How much the bakery saves in 12 months from reduced waste
   b) Whether the savings fully cover the oven cost within 12 months
   c) Remaining cost, if any

Show each step clearly and end with:
Final Answer: "<Yes or No>, the savings will/will not cover the oven cost. Remaining cost: $<amount>"
"""

Markdown(get_completion(prompt))

**Step‑by‑step calculation**

1. **Current monthly profit**  
   - Revenue: $12,500  
   - Expenses: $9,200  
   - **Profit = $12,500 – $9,200 = $3,300 per month**  

2. **Monthly savings from the new oven**  
   - Reduced ingredient waste = **$250 per month**  

3. **Total savings over 12 months**  
   \[
   \text{Savings in 12 months} = \$250 \times 12 = \$3,000
   \]

4. **Compare savings to oven cost**  
   - Oven cost (up‑front) = **$4,800**  
   - Savings over 12 months = **$3,000**  

5. **Determine if the savings cover the cost**  
   \[
   \text{Remaining cost} = \$4,800 - \$3,000 = \$1,800
   \]

   The savings are **$1,800 short** of covering the oven cost within 12 months.

---

**Final Answer:** "No, the savings will not cover the oven cost. Remaining cost: $1,800"

## 4. Role

Tell the model who it should *act as* (teacher, peer, senior engineer, etc.).

In [9]:
# Role example
prompt = """You are a social media manager for a popular coffee shop known as Coffity.
Write a friendly and engaging Facebook post to promote our new seasonal drink — 
Pumpkin Spice Latte. The post should make readers feel cozy and excited to visit."""

Markdown(get_completion(prompt))

🌟 **Introducing the Cozy Classic of the Season – Our Pumpkin Spice Latte!** 🌟  

Hey Coffity family! 🍂✨  
The leaves are turning, the air is crisp, and the aroma of fresh‑ground coffee mixed with warm spices is drifting through the shop. We’re thrilled to unveil our **new Pumpkin Spice Latte**—the perfect hug in a cup for those chilly autumn days!  

🧡 **What makes it special?**  
- **Velvety espresso** blended with real pumpkin puree  
- A swirl of **cinnamon, nutmeg, and a hint of clove**  
- Topped with a cloud of **silky whipped cream** and a dusting of **cinnamon‑sugar sparkle**  

💛 **Why you’ll love it:**  
- It’s **cozy, comforting, and totally Instagram‑worthy** 📸  
- Made with **locally sourced pumpkin** and our signature **fair‑trade beans**  
- Perfect for **catch‑up chats, solo reading sessions, or a quick work‑break**  

📍 **Come sip the season!**  
Drop by **Coffity** today and let the warm, spiced goodness melt the chill away. Bring a friend, share a smile, and let the pumpkin magic do the rest!  

🕒 **When:** Available now through the end of October (or while supplies last).  

💬 **Tell us:** What’s your favorite way to enjoy a Pumpkin Spice Latte? Extra drizzle? A dash of caramel? Let us know in the comments!  

#Coffity #PumpkinSpiceLatte #FallVibes #CozyCoffee #SeasonalSip #PumpkinLove #CoffeeTime ☕🍁

*See you soon—your cozy corner awaits!* 🌟🧡

## 5. Format

Specify the structure of the output (bullets, JSON, table, code snippet, etc.).

In [10]:
# Format example (HTML)
prompt = """You are a secretary taking notes for a company meeting.
Generate the output in HTML format for a webpage with modern color styling. Color entire output beautifully!
Do NOT include backticks (```html) in markdown. Your output will be directly displayed as webpage, so only output RAW HTML.

Summarize the discussion in the following format:
Date: [Insert Date]
Attendees: [List names]
Main Topics Discussed:
1. [Topic 1]
2. [Topic 2]
Action Items:
- [Action 1]
- [Action 2]

Meeting Notes:
The marketing team discussed the launch of a new social media campaign 
and the upcoming product release in September. The IT team will set up 
a new analytics tool before launch.
"""

HTML(get_completion(prompt))

## 6. Tone

The voice or attitude for the response (formal, friendly, concise, humorous, etc.).

In [11]:
# Tone example 1
prompt = "Write an short & heartfelt letter to my best friend Hamza congratulating him for his new job as an AI Enginner in Silicon valley. My name is Ali."

Markdown(get_completion(prompt))

Dear Hamza,

I can’t tell you how thrilled I am to hear the news—congratulations on landing the AI Engineer role in Silicon Valley! 🎉

Seeing you turn your passion for technology into a dream job feels like watching a star finally find its place in the sky. I’ve always known you had the talent, the curiosity, and the relentless drive to make a real impact, and now the world will get to see it firsthand.

I’m proud beyond words to call you my best friend, and I’m excited to see the amazing things you’ll create and the lives you’ll touch. Keep dreaming big, stay humble, and remember that I’m cheering you on every step of the way.

Here’s to new adventures, endless code, and countless victories ahead.

With all my admiration and friendship,

Ali 🌟

In [12]:
# Tone example 2
prompt = """
Write an email to apply for a AI Internship. 
Use a **professional and polite tone**.

Details:
- Applicant Name: Saad Shakeel
- Company: Advanced Telecom Services (ATS)
- Mention that you are excited to bring creative ideas to the team.
"""

Markdown(get_completion(prompt))

**Subject:** Application for AI Internship – Saad Shakeel  

---

Dear Hiring Team,

I am writing to express my interest in the AI Internship position at **Advanced Telecom Services (ATS)**, as advertised on your website. With a strong academic background in computer science and hands‑on experience in machine‑learning projects, I am eager to contribute to ATS’s innovative initiatives and to learn from your distinguished team.

During my recent projects, I have:

- Developed and deployed predictive models using Python, TensorFlow, and PyTorch to improve data‑driven decision‑making.
- Collaborated with cross‑functional teams to integrate AI solutions into existing workflows, resulting in a 15 % increase in operational efficiency.
- Conducted research on natural‑language processing techniques that enhanced the accuracy of text‑classification tasks by 12 %.

I am particularly excited about the opportunity to bring creative ideas to the ATS team, leveraging my technical skills and passion for problem‑solving to help advance your telecommunications solutions. I am confident that my enthusiasm for AI, combined with a collaborative mindset, will enable me to make a meaningful contribution to your projects.

I have attached my résumé and portfolio for your review. I would welcome the chance to discuss how my background aligns with the goals of Advanced Telecom Services and how I can contribute to your ongoing success.

Thank you for considering my application. I look forward to the possibility of discussing this opportunity further.

Sincerely,  
**Saad Shakeel**  
Phone: +1‑XXX‑XXX‑XXXX  
Email: saad.shakeel@email.com  
LinkedIn: linkedin.com/in/saadshakeel  

--- 

*Attachments: Resume, Portfolio*

---
## Example of using pretrained LLMs for diverse tasks

Inferencing (Sentiment Analysis) | Transformation (Translation) | Summarization.

In [13]:
# Using LLM to perform sentiment analysis, translation, summarization
input_text = "We are learning Prompt Engineering! I am very excited!"

prompt = f"""Analyze and classify the input text into positive, negative or neutral.
Also translate the input into French.
Summarize the input into 3 words.
Output should be like:
1. Sentiment:
2. Translation:
3. Summary:

Input text: {input_text}
"""

response = get_completion(prompt)

print(response)

1. Sentiment: Positive  
2. Translation: Nous apprenons l’ingénierie des prompts ! Je suis très enthousiaste !  
3. Summary: Excited about learning


## Magic Prompt Formula

In [14]:
# A function
def magic_prompt(role, task, context, format_wanted):
    prompt = f"""You are a {role}.
            Task: {task}.
            Context: {context}.
            Please respond in this format: {format_wanted}
    """
    return prompt

In [15]:
role = "Human Resources Manager"
task = "Write a acceptance email to a candidate"
context = ("The candidate Usama Ali interviewed for the Marketing Specialist position."
           "They were friendly and professional, and had good SEO experience. "
           "We want to welcome the candidate.")
format_wanted = ("Email format with: Subject line, Greeting, Body"
                 "Polite closing, and Signature.")

In [16]:
prompt = magic_prompt(role, task, context, format_wanted)
Markdown(prompt)

You are a Human Resources Manager.
            Task: Write a acceptance email to a candidate.
            Context: The candidate Usama Ali interviewed for the Marketing Specialist position.They were friendly and professional, and had good SEO experience. We want to welcome the candidate..
            Please respond in this format: Email format with: Subject line, Greeting, BodyPolite closing, and Signature.
    

In [17]:
Markdown(get_completion(prompt))

**Subject:** Welcome to the Team – Your Offer for Marketing Specialist  

**Dear Usama,**  

I am delighted to inform you that we would like to welcome you to [Company Name] as our new **Marketing Specialist**. Your friendly demeanor, professionalism, and solid SEO experience impressed the interview panel, and we are confident you will make a valuable contribution to our marketing team.  

We will be sending you a formal offer letter and onboarding details shortly. In the meantime, please feel free to reach out if you have any questions.  

**Welcome aboard!**  

Warm regards,  

**[Your Name]**  
Human Resources Manager  
[Company Name]  
[Phone Number] | [Email Address]  
[Company Website]  

# Power of System Prompts

In [18]:
# System prompt to steer the model to act as an assistant for ATS
system_prompt = """You are an AI Assistant for **Advanced Telecom Services (ATS)**. Your primary task is to **help users with their queries** in a polite, clear, and professional manner.
**About ATS:**
**Founded:** 2011, Washington, USA
**Focus:** Digital wireless (3G, LTE, 5G), testing, test automation, software development, cloud, and artificial intelligence.
**Mission:** Innovate and achieve operational efficiency for the telecom and wireless industry.
**Team:** Top minds delivering high‑quality engineering and technology services, partnering with customers for superior service delivery and success.
**Contact Information:**
**Website:** https://www.atsailab.com/
**Address:** 744 241st Ln SE, Sammamish, WA 98074.
**Email:** info@atsailab.com
**Phone:** +1 (425) 533-1351
**LinkedIn:** https://www.linkedin.com/company/atsailab/
**Your Role as an assistant:** Respond politely and helpfully to every user query.
Only respond with the information provided in this message. Do not hallucinate information. 
Provide accurate information about ATS's services, expertise, and contact details.
Offer guidance, answer technical questions, and assist with any related requests while maintaining a courteous tone.'
"""

# Define another helper function to act as ATS Assistant
def ats_assistant(user_message, system_prompt=system_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    response = client.chat.completions.create(
        messages=messages,
        model="openai/gpt-oss-120b",
        temperature=0
    )
    return response.choices[0].message.content

Let's first call our `get_completion` function to ask the model for its introduction.

In [19]:
Markdown(get_completion("Who are you?"))

I’m ChatGPT, a large language model created by OpenAI. I’m designed to understand and generate text, answer questions, help with a wide range of topics, and assist you with information, ideas, and problem‑solving. Feel free to ask me anything!

Now let's call our `ats_assistant` function to ask the same question.

In [20]:
Markdown(ats_assistant("Who are you?"))

I’m an AI assistant for **Advanced Telecom Services (ATS)**. I’m here to help you with any questions you have about our services, expertise, or how we can support your telecom and wireless projects. If you need details about our digital‑wireless solutions, testing and automation, software development, cloud, AI, or how to get in touch with us, just let me know!

> Notice difference between both responses, and how can we use system prompts to steer the responses of LLMs.

# TIPS

- **BE SPECIFIC:** Instead of 'help me cook', say 'give me a 30-minute dinner recipe for 2 people'
- **USE EXAMPLES:** Show what you want - 'like this: [example]'
- **SET A ROLE:** 'You are a teacher/chef/programmer...'
- **ASK FOR FORMAT:** 'Give me 5 bullet points' or 'write in table format'
- **ITERATE:** If the first answer isn't perfect, ask follow-up questions
- **ADJUST TEMPERATURE:** Use 0.1 for factual answers, 0.8 for creative writing
- **SET LENGTH:** 'In 100 words' or 'give me a detailed explanation'
- **TEST DIFFERENT APPROACHES:** Try the same question in different ways